# SEGMENTING AND CLUSTERING

# NEIGHBORHOODS IN TORONTO

# Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
print('Imported Successfully .....')

Imported Successfully .....


# Web Scraping for data

In [55]:
wiki='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wiki=requests.get(wiki).text


In [56]:
soup=BeautifulSoup(raw_wiki,'xml')


In [57]:
table=soup.find('table',{'class':'wikitable sortable'})


In [5]:
table_rows=table.find_all('tr')



# reading all rows from the table

In [6]:
data=[]
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])
df=pd.DataFrame(data,columns=['PostalCode','Borough','Neighbourhood'])
df=df[~df['PostalCode'].isnull()]

df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


# Cleaning data 

In [7]:
df.drop(df[df['Borough']=='Not assigned'].index,axis=0,inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [13]:
df1=df.reset_index()
df1.head()


,index,PostalCode,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,Regent Park / Harbourfront
3,6,M6A,North York,Lawrence Manor / Lawrence Heights
4,7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
index            103 non-null int64
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


In [47]:
df2=df1.groupby('PostalCode').agg(lambda x:','.join(x))
df2.head()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,Malvern / Rouge
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
M1E,Scarborough,Guildwood / Morningside / West Hill
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [48]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df2.head()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,Malvern / Rouge
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
M1E,Scarborough,Guildwood / Morningside / West Hill
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [49]:
df3 = df2.reset_index()
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Replace duplicates fromthe table

In [53]:
df3['Borough']= df3['Borough'].str.replace('nan/[{}\s]',' ').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

df3.groupby('Neighbourhood')
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [58]:
df3.shape

(103, 3)

In [70]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import geocoder

df4=df3.copy()
postal_codes=df3['PostalCode'].tolist()

for postal in postal_codes:
    lat_lng_coords=None
    print('{},Toronto, Ontario'.format(postal))
    
    while(lat_lng_coords is None):
        g=geocoder.google('{},Toronto, Ontario'.format(postal))
        lat_lng_coords=g.latlng
    
    latitude=lat_lng_coords[0]
    longitude=lat_lng_coords[1]
    
    df4.loc[df4.PostCode==postal,'Latitude']=latitute
    df4.loc[df4.PostCode==postal,'Longitude']=longitude
    
    

In [ ]:
df4